In [3]:
import os
import json5
import json
import math
import pandas as pd
import numpy as np
from datasets import load_dataset,Dataset

In [4]:
origin_xlsx = '/mnt/users/LLMa/ChatGLM-6B/data/MRA/头MRA数据-24个中心所有数据汇总-to深睿-1101.xlsx'
train_xlsx =  '/mnt/users/LLMa/ChatGLM-6B/data/MRA/东总血管疾病stl信息统计_20230220_3222例.xlsx'
val_xlsx = '/mnt/users/LLMa/ChatGLM-6B/data/MRA/东总血管疾病stl信息统计_1172例.xlsx'

In [5]:
origin_df = pd.read_excel(origin_xlsx,converters={'ID': str})
train_df = pd.read_excel(train_xlsx,converters={'ID': str})
val_df = pd.read_excel(val_xlsx,converters={'ID': str})

# origin_df['ID'] = origin_df['ID'].astype(str)
# train_df['ID'] = train_df['ID'].astype(str)
# val_df['ID'] = val_df['ID'].astype(str)

In [39]:
def get_pair(row,origin_df):
    # get original row
    pid = str(row['ID'])
    origin_row = origin_df.query("ID == '{}'".format(pid))
    
    if len(origin_row)==0:
        print(pid,'no origin report df')
        return None
        # continue
    
    res = ''
    hemo = row['MR是否有颅内出血？']
    count=1
    if '无出血' not in hemo:
        res+='{}.{};  '.format(count,hemo.strip())
        count+=1
    
    other = row['MR是否合并其他颅脑状况？']
    if '无' not in str(other):
        res+='{}.{};  '.format(count,str(other).strip())
        count+=1

    avm = row['是否存在动静脉畸形']
    if '无' not in avm:
        res+='{}.动静脉畸形;  '.format(count,avm.strip())
        count+=1
    
    for j in range(1,7):
        lesion = row['病灶{}'.format(j)]
        section = row['血管段{}'.format(j)]
        lr = row['左右{}'.format(j)]
        size = ''
        if pd.isna(lesion):
            lesion=''
        if pd.isna(section):
            section=''
        if pd.isna(lr):
            lr = ''
        if lesion+section+lr=='':
            continue
        elif '动脉瘤' in lesion:
            size = lesion.replace('#','')
            size = size.replace('＃','')
            size = size.split(':')[-1].strip()
            lesion = '动脉瘤'
            if len(size)>0:
                isdecimal=True
                try:
                    size = float(size)
                except ValueError:
                    isdecimal=False
                if isdecimal:
                    size = str(size)+'mm'
        if size!='':
            res+='{}.{}:左右:{}, 血管分段:{}, 大小:{};  '.format(count,lesion,lr,section,size)
        else:
            res+='{}.{}:左右:{}, 血管分段:{};  '.format(count,lesion,lr,section)
        count+=1
   
    if len(res) == 0:
        res = '颅脑MR未见异常'
    item0 = list(origin_row['MR所见'])[0]
    if pd.isna(item0):
        item0=''
    item1 = list(origin_row['MR报告'])[0]
    if pd.isna(item1):
        item1=''
    mr_content = item0 #+'\n'+item1
    if len(mr_content)==0:
        print(pid)
    pair = {'Input':mr_content,'Output':res}
    return pair

In [40]:
train_info=[]
for idx,row in train_df.iterrows():
    pair = get_pair(row,origin_df)
    if pair is not None:
        train_info.append(pair)

p01287091 no origin report df
298386
571120
677898
712321
742682
757422
613937
00202609
00227851
00238140
00258355
00272197
00299629
00304793
00324744
00334281
00334522
00356814
00370793
00416120
00429807
00450534
00452578
00470003
00483542
00582043
00609959
00611623
00636491
00669169
00675693
00684761
00702744
00704778
00729125
259158
285508
491035
491390
622235
673899
686199
686665
687092
711032
724326
741497
759963
780133
784106
829363
852292
893724
896458
917227
927070
995839
1002198
1029923
1045275
1057544
1094148
1098615
1104591
1105556
1132744
1193787
1213082
00400724
00400202
00408770
00409067
00410471
00413595
00415720
00420679
00421382
00436878
00449970
00453234
00454999
00455550
00457864
00460581
00460881
00471547
1806220168
1806240021
1806280176
p1806260019
p1806260019
MR298402
00170194
00177642
00179251
00213760
00220974
00230126
00242783
00244045
00244299
00252668
00255182
00262242
00275892
00275920
00285460
00289858
00300136
00301878
00303005
00306408
00309600
00312371
0

In [32]:
train_data = Dataset.from_list(train_info)
print(len(train_data))
# train_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/MRA/train.csv')
train_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/MRA/train.json')

3221


Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

3715706

In [33]:
val_info=[]
for idx,row in val_df.iterrows():
    pair = get_pair(row,origin_df)
    if pair is not None:
        val_info.append(pair)

In [34]:
val_data = Dataset.from_list(val_info)
print(len(val_data))
# val_data.to_csv('/mnt/users/LLMa/ChatGLM-6B/data/MRA/val.csv')
val_data.to_json('/mnt/users/LLMa/ChatGLM-6B/data/MRA/val.json',force_ascii=False)

1171


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

1023064